In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Чтобы визуализировать работу алгоритмов машинного обучения, часто бывает полезно изучить двумерные или одномерные данные, то есть данные, имеющие только один или два признака. Хотя на практике наборы данных обычно имеют гораздо больше признаков, при этом отображать многомерные данные на двумерных экранах достаточно сложно.

Проиллюстрируем несколько очень простых примеров, прежде чем перейдем к более «реальным» наборам данных.

Классификация  Classification
========
Сначала мы рассмотрим задачу классификации двух классов в двух измерениях. Мы используем синтетические данные, сгенерированные функцией `make_blobs`.

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(centers=2, random_state=0)
print(X.shape)
print(y.shape)
print(X[:5, :])
print(y[:5])

Поскольку данные двумерны, мы можем отобразить каждый образец как точку в двумерном пространстве, где первый признак — это ось X, а второй — ось Y.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plt.show()

Поскольку классификация — это задача обучения с учителем, и нас интересует, насколько хорошо модель обобщает, мы разделяем наши данные на обучающий набор для построения модели и тестового набора для оценки, насколько хорошо наша модель работает на новых данных. Функция `train_test_split` из модуля `model_selection` делает это за нас, случайным образом отделяя 25% данных для тестирования.

<img src="figures/train_test_split.svg" width="100%">


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### The scikit-learn estimator API
<img src="figures/supervised_workflow.svg" width="100%">


Каждый алгоритм представлен в scikit-learn через объект из соответствующего класса с родительским классом ''Estimator'' (Оценка).

Например, логистическая регрессия реализуется через класс `LogisticRegression`:

In [ ]:
from sklearn.linear_model import LogisticRegression

Все модели в scikit-learn имеют единый интерфейс с последовательным пошаговым применением методов.

Сначала мы создаем экземпляр оценивающего класса.

In [ ]:
classifier = LogisticRegression()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

Чтобы построить модель на основе наших данных, то есть научиться классифицировать новые точки, мы вызываем функцию `fit` («подгонки») с обучающими данными и соответствующими обучающими метками (желаемый результат для обучающей точки данных):

In [ ]:
classifier.fit(X_train, y_train)

Затем мы можем применить модель к неизвестным ей данным и использовать её для прогнозирования предполагаемого результата с помощью метода прогнозирования `predict`:

In [ ]:
prediction = classifier.predict(X_test)

Мы можем сравнить их с реальными метками классов:

In [ ]:
print(prediction)
print(y_test)

Мы можем оценить наш классификатор количественно, определив, какая часть прогнозов верна. Это называется оценкой **точности** (**accuracy**):

In [ ]:
np.mean(prediction == y_test)

Существует также удобная функция `score`, которую все классификаторы scikit-learn должны вычислять непосредственно на основе тестовых данных:

In [ ]:
classifier.score(X_test, y_test)

Часто бывает полезно сравнить эффективность обобщения (на тестовом наборе) с эффективностью на обучающем наборе:

In [ ]:
classifier.score(X_train, y_train)

Дополнительные средства оценивания результатов обучения:

In [ ]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, prediction))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

Логистическая регрессия — это так называемая линейная модель, а это означает, что создаётся решение, линейное во входном пространстве. В 2d это просто означает, что модель находит линию, отделяющую синие метки от красных:

In [ ]:
def plot_2d_separator(classifier, X, fill=False, line=True, ax=None, eps=None):
    if eps is None:
        eps = 1.0 #X.std() / 2.
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 100)
    yy = np.linspace(y_min, y_max, 100)

    X1, X2 = np.meshgrid(xx, yy)

    X_grid = np.c_[X1.ravel(), X2.ravel()]

    try:
        decision_values = classifier.decision_function(X_grid)
        levels = [0]
        fill_levels = [decision_values.min(), 0, decision_values.max()]

    except AttributeError:
        # no decision_function
        decision_values = classifier.predict_proba(X_grid)[:, 1]
        levels = [.5]
        fill_levels = [0, .5, 1]
    if ax is None:
        ax = plt.gca()
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape), levels=fill_levels, colors=['cyan', 'pink'])
    if line:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels, colors="black")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

In [ ]:
#from figures import plot_2d_separator

plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(classifier, X)
plt.show()

**Оценочные параметры**. Все расчетные параметры являются атрибутами объекта оценки, оканчивающимися знаком подчеркивания. Для логистической регрессии это коэффициенты и смещение линии:

In [ ]:
print(classifier.coef_)
print(classifier.intercept_)

Другой классификатор: K Nearest Neighbors (K ближайших соседей).
--------------------------------------------------
Другой популярный и простой для понимания классификатор — K ближайших соседей K nearest neighbors (kNN). Он имеет одну из самых простых стратегий обучения: учитывая новое, неизвестное наблюдение, находятся среди других данных те наблюдения, которые имеют наиболее близкие признаки, и назначается преобладающий класс.

Интерфейс точно такой же, как и для `LogisticRegression` рассмотреного выше.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

На этот раз используется параметр для KNeighborsClassifier, чтобы указать, что мы хотим просмотреть только одного ближайшего соседа:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

Обучаем модель на обучающих данных:

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(knn, X)
plt.show()

In [ ]:
knn.score(X_test, y_test)

И посмотрим дополнительные средства оценивания результатов обучения:

In [ ]:
from sklearn.metrics import confusion_matrix

knn_prediction = knn.predict(X_test)

print (confusion_matrix(y_test, knn_prediction))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, knn_prediction))

Что будет, если использовать при обучении два соседа, три...

Попробуйте изменить параметр `n_neighbors`

И ещё классификатор: Random Forest  (Случайное дерево).
--------------------------------------------------

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=2) #, n_estimators=100)

clf.fit(X_train, y_train)
clf_prediction = clf.predict(X_test)
print(classification_report(y_test, clf_prediction))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(clf, X)
plt.show()

Упражнение
============
Примените KNeighborsClassifier и/или RandomForestClassifier к набору данных `iris`. Поиграйте с разными значениями `n_neighbors` и понаблюдайте, как меняются результаты обучения и теста.